In [6]:
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import os 

model_folder = r'C:\Users\yucheng_chih\Desktop\self_atten\model_foder'
os.makedirs(model_folder, exist_ok=True)
# Load data
df = pd.read_csv(r"C:\Users\yucheng_chih\Desktop\self_atten\question_paragraph_1比2.csv")
label = df['label'].to_numpy()
load_np1 = np.load(r"C:\Users\yucheng_chih\Desktop\self_atten\question2answer-sen1_vector.npz")
vector1_ = load_np1['sen1']
load_np2 = np.load(r"C:\Users\yucheng_chih\Desktop\self_atten\question2answer-sen2_vector.npz")
vector2_ = load_np2['sen1']

# Split data
train_vector1, test_vector1, train_vector2, test_vector2, train_y, test_y = train_test_split(
    vector1_, vector2_, label, test_size=0.1, random_state=42)

# Convert to PyTorch tensors
train_vector1 = torch.tensor(train_vector1, dtype=torch.float32)
#train_vector1 = train_vector1/torch.max(train_vector1)
train_vector2 = torch.tensor(train_vector2, dtype=torch.float32)
#train_vector2 = train_vector2/torch.max(train_vector2)
#向量預處理
final_vector_train = (torch.cat((train_vector1, train_vector2), dim=-1).squeeze(1))
#向量預處理
train_y = torch.tensor(train_y, dtype=torch.long)

test_vector1 = torch.tensor(test_vector1, dtype=torch.float32)
#test_vector1 = test_vector1/torch.max(test_vector1)
test_vector2 = torch.tensor(test_vector2, dtype=torch.float32)
#test_vector2 = test_vector1/torch.max(test_vector2)
#向量預處理
final_vector_test = (torch.cat((test_vector1, test_vector2), dim=-1).squeeze(1))
#向量預處理
test_y = torch.tensor(test_y, dtype=torch.long)

# Create TensorDatasets
trainsets = TensorDataset(final_vector_train, train_y)
testsets = TensorDataset(final_vector_test, test_y)

# Create DataLoaders
trainloader = DataLoader(trainsets, batch_size=4, shuffle=True, num_workers=0)
testloader = DataLoader(testsets, batch_size=4, shuffle=False, num_workers=0)



In [7]:
import torch.nn as nn
import torch

class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x):
        # 計算 Q, K, V 矩陣
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        # 計算注意力分數並應用數值穩定化技術
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / (x.size(-1) ** 0.5)
        attention_scores = attention_scores - attention_scores.max(dim=-1, keepdim=True)[0]  # 減去最大值
        attention_weights = self.softmax(attention_scores)  # 單次 softmax 操作
        
        # 計算注意力輸出
        attention_output = torch.matmul(attention_weights, V)
        
        return attention_output

class AttentionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(AttentionModel, self).__init__()
        self.self_attention1 = SelfAttention(input_dim)
        self.self_attention2 = SelfAttention(input_dim)
        self.self_attention3 = SelfAttention(input_dim)
        
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.self_attention1(x)
        x = self.relu(x)
        x = self.self_attention2(x)
        x = self.relu(x)
        x = self.self_attention3(x)
        x = self.relu(x)
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)


In [3]:
# import torch.nn as nn

# class AttentionModel(nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super(AttentionModel, self).__init__()
#         self.self_attention1 = SelfAttention(input_dim)
#         self.batch_norm1 = nn.BatchNorm1d(input_dim)
        
#         self.self_attention2 = SelfAttention(input_dim)
#         self.batch_norm2 = nn.BatchNorm1d(input_dim)
        
#         self.self_attention3 = SelfAttention(input_dim)
#         self.batch_norm3 = nn.BatchNorm1d(input_dim)
        
#         # 定義全連接層，並增加 ReLU 激活函數
#         self.fc1 = nn.Linear(input_dim, 2048)
#         self.fc2 = nn.Linear(2048, 1024)
#         self.fc3 = nn.Linear(1024, 512)
#         self.fc4 = nn.Linear(512, 256)
#         self.fc5 = nn.Linear(256, 128)
#         self.fc6 = nn.Linear(128, 32)
#         self.fc7 = nn.Linear(32, 4)
#         self.fc8 = nn.Linear(4, num_classes)

#         self.relu = nn.ReLU()
    
#     def forward(self, x):
#         # 過濾多層 Self-Attention 層，並在每層之間應用批次正規化
#         x = self.self_attention1(x)
#         x = self.batch_norm1(x)
#         x = self.relu(x)
        
#         x = self.self_attention2(x)
#         x = self.batch_norm2(x)
#         x = self.relu(x)
        
#         x = self.self_attention3(x)
#         x = self.batch_norm3(x)
#         x = self.relu(x)
        
#         # 全連接層，逐層應用 ReLU 激活函數
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.relu(self.fc3(x))
#         x = self.relu(self.fc4(x))
#         x = self.relu(self.fc5(x))
#         x = self.relu(self.fc6(x))
#         x = self.relu(self.fc7(x))
#         x = self.fc8(x)  # 最後一層不使用 ReLU
        
#         return x

In [8]:

lr = 1e-4
epochs = 10
epoch_save_test=2
model = AttentionModel(input_dim=final_vector_train.size(1), num_classes=2)
model.apply(initialize_weights)
weight = torch.tensor([1.0,2.0],dtype=torch.float32)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)



In [9]:
import datetime

for epoch in range(epochs):
    print("epoch:", epoch)
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    count = 0
    try:
        for i, data in enumerate(trainloader, 0):
                
            inputs, labels = data
            if torch.isnan(inputs).any() or torch.isinf(inputs).any():
                print("NaN or Inf found in inputs")
            if torch.isnan(labels).any() or torch.isinf(labels).any():
                print("NaN or Inf found in labels")
                count += 1
                print(f"Processing batch {count}")
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            print(loss)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.25)            
            optimizer.step()
            running_loss += loss.item()

            _, predicted_train = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted_train == labels).sum().item()
            # for name, param in model.named_parameters():
            #     if param.grad is not None:
            #         print(f"{name}: min grad {param.grad.min()}, max grad {param.grad.max()}")

        avg_train_loss = running_loss / len(trainloader)
        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    except Exception as e:
        print(f"Error occurred in epoch {epoch+1}: {e}")
        break
  

    # 測試模型
    model.eval()
    correct = 0
    total = 0
    running_val_loss = 0.0
    if (epoch + 1) % epoch_save_test == 0:
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                outputs = model(inputs)
                loss = criterion(outputs, labels)  # 計算驗證損失
                running_val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = running_val_loss / len(testloader)  # 驗證損失平均值
        accuracy = 100 * correct / total
        print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy on test set: {accuracy:.2f}%")
    
    # 儲存模型每25個epoch
    
    if (epoch + 1) % epoch_save_test == 0:
        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        model_name = (f"model_{current_time}_epoch{epoch+1}_"
                      f"batchsize{trainloader.batch_size}_lr{optimizer.param_groups[0]['lr']}_"
                      f"train_loss{avg_train_loss:.4f}_train_acc{train_accuracy:.2f}_"
                      f"val_loss{avg_val_loss:.4f}_test_acc{accuracy:.2f}.pth")
        model_path = os.join.path(model_folder, model_name )
        torch.save(model.state_dict(), model_path)
        print(f"Model saved as {model_path}")

epoch: 0
tensor(0.9249, grad_fn=<NllLossBackward0>)
tensor(22.4123, grad_fn=<NllLossBackward0>)
tensor(0.9372, grad_fn=<NllLossBackward0>)
tensor(12.8791, grad_fn=<NllLossBackward0>)
tensor(0., grad_fn=<NllLossBackward0>)
tensor(7.8436, grad_fn=<NllLossBackward0>)
tensor(0.9074, grad_fn=<NllLossBackward0>)
tensor(3.0431, grad_fn=<NllLossBackward0>)
tensor(10.4731, grad_fn=<NllLossBackward0>)
tensor(13.2856, grad_fn=<NllLossBackward0>)
tensor(4.0666, grad_fn=<NllLossBackward0>)
tensor(3.5883, grad_fn=<NllLossBackward0>)
tensor(3.1897, grad_fn=<NllLossBackward0>)
tensor(16.9046, grad_fn=<NllLossBackward0>)
tensor(15.2614, grad_fn=<NllLossBackward0>)
tensor(2.9636, grad_fn=<NllLossBackward0>)
tensor(6.7963, grad_fn=<NllLossBackward0>)
tensor(15.1646, grad_fn=<NllLossBackward0>)
tensor(6.3492, grad_fn=<NllLossBackward0>)
